### This colab file augment QA dataset questions from sources of **governance-v2**, **whitepaper** and **yield space papers**. It also augment previous document sources via custom prompting to **docs-v2** and **addendum-docs**.

### This colab file also experiments with integrated QA-generation pipeline to ensure a smooth and trustworthy data generation flow.

In [ ]:
# Install dependencies
!pip install langchain===0.0.230 openai chromadb==0.3.26 pydantic==1.10.8 GitPython ipython tiktoken
!pip install unstructured==0.8.5 pdf2image pytesseract pypdf
!apt-get install -y poppler-utils tesseract-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.4 MB/s eta 0:00:00
  Attempting uninstall: unstructured
    Found existing installation: unstructured 0.6.5
    Uninstalling unstructured-0.6.5:
      Successfully uninstalled unstructured-0.6.5


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [ ]:
import os
from langchain.document_loaders import GitLoader, UnstructuredPDFLoader, OnlinePDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQAWithSourcesChain, RetrievalQA
from IPython.display import display
from IPython.display import Markdown
from getpass import getpass
from pathlib import Path
from langchain.callbacks import StdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
import json

stdout_handler = StdOutCallbackHandler()

In [ ]:
OPENAI_API_KEY = getpass()

··········


In [ ]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

## Load up repo and pdf documents and generate questions

In [ ]:
# Load repository with .md files as targets to split and store
def load_repo(remote_repo_url, local_repo_path, branch, file_filter=None):
    local_repo_exists = Path(local_repo_path).is_dir() # second and later loadings

    if local_repo_exists:
        loader = GitLoader(
            repo_path=local_repo_path,
            branch=branch,
            file_filter=file_filter
        )
    else:
        loader = GitLoader(
            clone_url=remote_repo_url,
            repo_path=local_repo_path,
            branch=branch,
            file_filter=file_filter
        )
    return loader.load() # load the required source files

# Load PDF documents
def load_pdf(file_path):
    loader = UnstructuredPDFLoader(file_path)
    return loader.load()

def load_online_pdf(file_url):
    loader = OnlinePDFLoader(file_url)
    return loader.load()

# Load single md document from drive/filesystem
def load_md(markdown_path):
    loader = TextLoader(markdown_path) # raw, do not parse md. https://github.com/langchain-ai/langchain/issues/3591
    return loader.load()

def split_docs(docs, language, chunk_size, chunk_overlap, force_category):
    text_splitter = RecursiveCharacterTextSplitter.from_language(language=language, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    all_splits=[]
    all_metadatas=[]
    for d in docs:
        doc_file = d.page_content
        metadata = d.metadata # metadata including filepath, filename, other stuffs
        splits = text_splitter.split_text(doc_file) # parse the document to small chunk of code snippets

        metadata['category'] = force_category

        # this is only for governance type questions
        if metadata['source'].split('.')[-1] == 'md':
          title = f"# {metadata['file_name'].split('.')[0]}"
          for i in range(len(splits)):
            splits[i] = f"{title}\n\n{splits[i]}" # string is immutable, must write explitly


        metadatas = [metadata for _ in splits]
        all_splits += splits
        all_metadatas += metadatas # collecting metadata headers and the splited chunks from each document.

    return {
        'all_splits': all_splits,
        'all_metadatas': all_metadatas # pack as dict/json
    }

def split_pdf(docs, chunk_size, chunk_overlap, force_category):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap) # no language as pdf is parsed as plain txt

    all_splits=[]
    all_metadatas=[]
    for d in docs:
        doc_file=d.page_content
        metadata = d.metadata # metadata including filepath, filename, other stuffs
        splits = text_splitter.split_text(doc_file) # parse the document to small chunk of code snippets

        metadata['category'] = force_category

        metadatas = [metadata for _ in splits]
        all_splits += splits
        all_metadatas += metadatas # collecting metadata headers and the splited chunks from each document.

    return {
        'all_splits': all_splits,
        'all_metadatas': all_metadatas # pack as dict/json
    }

In [ ]:
from langchain.text_splitter import MarkdownTextSplitter
def split_md(docs, chunk_size, chunk_overlap, force_category):
    text_splitter = MarkdownTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

    all_splits=[]
    all_metadatas=[]
    for d in docs:
        doc_file = d.page_content
        metadata = d.metadata # metadata including filepath, filename, other stuffs
        splits = text_splitter.split_text(doc_file) # parse the document to small chunk of code snippets

        metadata['category'] = force_category

        metadatas = [metadata for _ in splits]
        all_splits += splits
        all_metadatas += metadatas # collecting metadata headers and the splited chunks from each document.

    return {
        'all_splits': all_splits,
        'all_metadatas': all_metadatas # pack as dict/json
    }

In [ ]:
# documentation repo
remote_repo_docsV2_url="https://github.com/yieldprotocol/docs-v2" # the docs all coming from the github repo (public)
local_repo_docsV2_path="/tmp/yield_docs_v2_repo"

# cookbook repo
remote_repo_addendum_url="https://github.com/yieldprotocol/addendum-docs"
local_repo_addendum_path="/tmp/yield_addendum-docs"

# governance repo, the documents under this repo is kinda problematic, does not have a lot of content, most are link to external files
# thinking of whether to load the files affected by governance-v2 as well?
remote_repo_governance_url = "https://github.com/yieldprotocol/governance-v2"
local_repo_governance_path = "/tmp/governance-v2"

branch="main" # specify the branch of the remote repo urls
file_filter=lambda file_path: file_path.endswith(".md") # select only .md files as 'documentations'

chunk_size_chars = 2000 # context length required, in SFT chunk it to 1000 at evaluation
chunk_overlap_chars = 0 # no overlapping since containing code chunk

documentation_docs = load_repo(remote_repo_docsV2_url, local_repo_docsV2_path, branch, file_filter)
addendum_docs = load_repo(remote_repo_addendum_url, local_repo_addendum_path, branch, file_filter)
governance_docs = load_repo(remote_repo_governance_url, local_repo_governance_path, branch, file_filter)

documentation_splits = split_docs(documentation_docs, Language.MARKDOWN, chunk_size_chars, chunk_overlap_chars, force_category='general')
addendum_splits = split_docs(addendum_docs, Language.MARKDOWN, chunk_size_chars, chunk_overlap_chars, force_category='technical')
governance_splits = split_docs(governance_docs, Language.MARKDOWN, chunk_size_chars, chunk_overlap_chars, force_category='governance')

# trim down first and last file of gov_splits, as there are placeholders
governance_splits['all_splits'] = governance_splits['all_splits'][1:]
governance_splits['all_metadatas'] = governance_splits['all_metadatas'][1:]

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from google.colab import drive
drive.mount('/content/drive/')

whitepaper_data = load_online_pdf(file_url="https://yieldprotocol.com/Yield.pdf")
yieldspace_data = load_md(markdown_path="/content/drive/MyDrive/YieldSpace.md")

# sanitize the source
whitepaper_data[0].metadata['source'] = "https://yieldprotocol.com/Yield.pdf"
yieldspace_data[0].metadata['source'] = "https://yieldprotocol.com/YieldSpace.pdf"

whitepaper_splits = split_pdf(whitepaper_data, 2000, chunk_overlap_chars, force_category="paper")
yieldspace_splits = split_md(yieldspace_data, 2500, 0, force_category="paper")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' not found
[nltk_data]     in index
[nltk_data] Error loading tokenizers: Package 'tokenizers' n

In [ ]:
for i, split in enumerate(whitepaper_splits['all_splits']):
  display(Markdown(f"# Chunk {i+1}"))
  display(Markdown(split))

# Chunk 1

The Yield Protocol: On-Chain Lending With Interest Rate Discovery

Dan Robinson

Allan Niemerg

dan@paradigm.xyz

allan@yield.is

April 2020 WORKING DRAFT, rev. 2

Abstract

This paper presents a sketch of a new building block for decentralized ﬁnance: fyTokens. fyTokens are like zero-coupon bonds: on-chain obli- gations that settle on a speciﬁc future date based on the price of some target asset, and are secured by collateral in another asset. By buying or selling fyTokens, users can synthetically lend or borrow the target asset for a ﬁxed term. fyTokens are fungible and trade at a ﬂoating price, which means their “interest rates” are determined by the market. The prices of fyTokens of varying maturities can be used to infer interest rates, and even to construct a yield curve. Depending on the target asset, fyTokens can settle through “cash-settlement” using an on-chain price oracle, through “physical settlement” in the target ERC20 token, or by synthetically is- suing or borrowing the target ERC20 token on another platform.

1

Introduction

The Yield Protocol is a standard for a token that settles based on the value of a target asset on a speciﬁed future date, and which is backed by some quantity of a collateral asset. We call these tokens fyTokens,1 or “ﬁxed yield tokens.”

You can create fyTokens by depositing collateral, then sell them to eﬀectively borrow (and short) the target asset. Buying fyTokens is economically similar to lending the target asset. The eﬀective “interest rate” received by fyToken holders is determined by the discount at which fyTokens currently trade, as well as the time to maturity.

fyTokens can also be used as a building block for more sophisticated prod- ucts. While each maturity of fyTokens has a ﬁxed expiration date, you could

1This nomenclature is inspired by the “cTokens” used in the Compound Protocol [1].

Previously released versions of this paper used the term “yTokens.”

1

# Chunk 2

build a perpetual product on top of it, by implementing a pool that invests in short-term fyTokens and automatically rolls them over upon expiry.

Perhaps most interestingly, the market prices of fyTokens can be used as an “interest rate oracle.” The price of each maturity of fyToken implies a partic- ular yield. These implied yields could be used to settle on-chain interest rate derivatives, to inform Maker’s choice of stability fee, or as an input to the in- terest rate formulas used by Compound or dYdX. By charting those yields for diﬀerent maturities, we can even construct a yield curve, which could be a useful indicator of the expected path of interest rates or prices.

This paper will present three constructions of fyTokens. The ﬁrst (“cash settlement”) assumes the existence of a price oracle that, when asked, can tell the contract the value of the target asset in terms of the collateral asset. The second (“physical settlement”) does not require any price oracle, but assumes that the target asset is itself a token (rather than an oﬀ-chain asset like USD). The third (settlement to synthetics) assumes that the target asset is either a collateralized synthetic like Maker, or supported by a ﬂoating-rate lending platform like Compound.

2 Prior work

The design of the Yield Protocol is heavily inﬂuenced by other projects on the Ethereum blockchain that oﬀer synthetics, borrowing, lending, and/or leverage. Yield primarily diﬀers in that its interest rates are implicit and set by market prices, rather than being set by governance or a formula. Additionally, whereas most lending protocols use ﬂoating interest rates, Yield enables term loans with ﬁxed interest rates, while still maintaining some degree of fungibility. Section 4.3 shows how, thanks to these properties, the Yield Protocol can provide unique insight into the term structure of interest rates for on-chain lending.

# Chunk 3

...

There are some optimizations that could reduce the number of calls to the oracle (although they may require setting larger collateralization requirements to maintain the same level of safety). To sketch out some of them:

When a vault is liquidated, instead of calling the price oracle, the liquida- tor could put up a security deposit. The vault’s owner can then cancel the liquidation and claim the security deposit by calling the oracle and proving that the vault was fully collateralized. If the liquidation has not been cancelled within 24 hours, the liquidation completes.

When fyTokens are minted, the creator could either call the price oracle,

or choose one of the following alternatives:

– Point to an existing vault for the same fyToken with an equal or lower liquidation price, that is not in the process of being liquidated. The existence of such a vault implies that the liquidation price has not been reached.

– Initiate a withdrawal of new fyTokens, then wait 24 hours. During that waiting period, anyone can initiate a liquidation of the position if it is undercollateralized (based on the post-withdrawal quantity of collateral). Once the waiting period is over, if no such liquidation is in progress, the withdrawal completes.

6 Future work

As mentioned in section 4.2, there are likely some interest rate derivatives that could use fyToken prices as an oracle. These might be useful for expressing a view on the future path of interest rates, or for hedging duration while holding fyTokens or owning a vault.

The physical settlement system described in section 3.3 could likely be adapted to create Bitcoin futures on Ethereum, adapting ideas from Summa

13

[16] and tBTC [17] (such as using an SPV-proof-based orderbook for a price or- acle for proof of undercollateralization, and using SPV proofs to enforce physical settlement).

7 Acknowledgments

# Chunk 20

This paper is heavily indebted to discussions with Hayden Adams, Arthur Breitman, Vitalik Buterin, Jill Carlson, Karl Floersch, Alex Herrmann, Ben Jones, Martin K¨oppelmann, Zubin Koticha, Aparna Krishnan, Hart Lambur, Teo Leibowitz, Robert Leshner, Lev Livnev, Allison Lu, Matt Luongo, James Prestwich, Cyrus Younessi, and Noah Zinsmeister. The authors are particularly grateful to Paradigm for supporting this research.

References

[1] Robert Leshner and Geoﬀrey Hayes. Compound: The Money Market Pro- tocol. Feb. 2019. url: https://compound.finance/documents/Compound. Whitepaper.pdf.

[2] MakerDAO. The Dai Stablecoin System. url: https://makerdao.com/

en/whitepaper/.

[3] Synthetix Litepaper v1.1. Mar. 2019. url: https://www.synthetix.io/

uploads/synthetix_litepaper.pdf.

[4] Hart Lambur. UMA – Universal Market Access. Dec. 2018. url: https: / / medium . com / uma - project / uma - enabling - universal - market - access-266eb9e5fd90.

[5] Dan Robinson. Rainbow Network: An Oﬀ-Chain Decentralized Synthetics

Exchange. Mar. 2019. url: http://research.paradigm.xyz/RainbowNetwork. pdf.

[6] Kyle J. Kistner. Introducing Fulcrum: Tokenized Margin Made Dead Sim- ple. Mar. 2019. url: https://medium.com/bzxnetwork/introducing- fulcrum-tokenized-margin-made-dead-simple-e65ccc82393f. [7] Antonio Juliano. Introducing the new dYdX. url: https://medium.com/

dydxderivatives/introducing-the-new-dydx-9675719bacb6.

[8] Nadav Hollander. Dharma: An Open Protocol for Generic Tokenized Debt Agreements. url: https://blog.dharma.io/dharma-an-open-protocol- for-generic-tokenized-debt-agreements-9a4e6a4e6fc0.

[9] MakerDAO. Vaults. url: https://community-development.makerdao.

com/makerdao-mcd-faqs/faqs/vault.

[10] Hayden Adams. Uniswap. url: https://uniswap.org/docs/. [11] Coinbase. USD Coin (USDC). url: https://www.coinbase.com/usdc.

14

[12] Noah Zinsmeister Hayden Adams and Dan Robinson. Uniswap v2 Core.

url: https://uniswap.org/whitepaper.pdf/.

# Chunk 21

[13] Fernando Martinelli and Nikolai Mushegian. A non-custodial portfolio manager, liquidity provider, and price sensor. url: https://balancer. finance/whitepaper/.

[14] Set: A Protocol for Baskets of Tokenized Assets. url: https : / / www .

setprotocol.com/pdf/set_protocol_whitepaper.pdf.

[15] UMA Data Veriﬁcation Mechanism: Adding Economic Guarantees to Blockchain Oracles. July 2019. url: https://github.com/UMAprotocol/whitepaper/ blob/master/UMA-DVM-oracle-whitepaper.pdf.

[16] Summa. Welcome to Summa Auctions. url: https://summa.one/auction. tBTC: A Decentralized Redeemable BTC-backed ERC-20 Token. Aug. 2019. [17] url: http://docs.keep.network/tbtc/index.pdf.

8 Disclaimer

This paper is for general information purposes only. It does not constitute investment advice or a recommendation or solicitation to buy or sell any in- vestment and should not be used in the evaluation of the merits of making any investment decision. It should not be relied upon for accounting, legal or tax advice or investment recommendations. This paper reﬂects current opinions of the authors and is not made on behalf of Paradigm or its aﬃliates and does not necessarily reﬂect the opinions of Paradigm, its aﬃliates or individuals as- sociated with Paradigm. The opinions reﬂected herein are subject to change without being updated.

15

In [ ]:
vector_db = Chroma(embedding_function=OpenAIEmbeddings())
# chunking them already
documentation_splits_ids = vector_db.add_texts(documentation_splits['all_splits'], documentation_splits['all_metadatas'])
addendum_splits_ids = vector_db.add_texts(addendum_splits['all_splits'], addendum_splits['all_metadatas'])
governance_splits_ids = vector_db.add_texts(governance_splits['all_splits'], governance_splits['all_metadatas'])
whitepaper_splits_ids = vector_db.add_texts(whitepaper_splits['all_splits'], whitepaper_splits['all_metadatas'])
yieldspace_splits_ids = vector_db.add_texts(yieldspace_splits['all_splits'], yieldspace_splits['all_metadatas'])

In [ ]:
# You are a Web3 user who wants to learn about the Yield protocol and its inner workings, concepts and methods to integrate using both client-side and smart contract code.

question_bank_prompt_template = """
You are an inquisitive Web3 user who wants to learn about the YieldSpace paper released by Yield Protocol that explains the mathematical foundations of YieldSpace.

# INSTRUCTIONS
- Generate 5 questions to ask about the protocol from the document provided below, always ensure the questions are related to the document text and do not make up any information.
- When you ask a question, make sure that you can also find evidences in the document that makes it answerable, but do not attempt to answer.
- The questions should be in plain text as much as possible, when the question contains latex formula, write the latex symbols in double slash '\\\\' to avoid json decoder error.
- Organize the results in lines, each question takes a line. Do not enumerate the questions, just write them to lines.

# DOCUMENT
{document}

# RESULT"""

QUESTION_BANK_PROMPT = PromptTemplate(
    template=question_bank_prompt_template, input_variables=["document"]
)

# one single document (chunk) and asking 5 questions
# 16k context is also available.
question_bank_llm_chain = LLMChain(
    llm=ChatOpenAI(temperature=0.10, model="gpt-4"),
    prompt=QUESTION_BANK_PROMPT
)

# code-related hallucination --> maybe turned it off when the first tested finetuned LLAMA-V2 is tested out.

answer_prompt_template = """
You are a Web3 expert who is able to answer any user query on Yield protocol's documentation, code, whitepapers and many other such topics.

# INSTRUCTIONS
- The user's query will be wrapped in triple back ticks
- Only answer the query using the provided context below which may include general information and code, do not make up any information.
- If you want to explicitly support your answer from the context, quote the part in the document in your answer.
- If the query is related to code or integration, provide a step by step explanation on the process, use code suggestions whereever relevant and always use markdown format annotated with the language to show the code.
- If the query is related to mathematic formulas or mathematic proofs, explain in natural languguage in an easy-to-understand manner and use markdown/latex to compose math formulas if applicable.
- For code suggestions, use comments to explain each important concept, class, variable, function and parameter.
- However, if you could not find documented code reference in the context, do not attempt to deliniate your answer in code, instead, use natural language supported from the context retrieved.
- Yield Protocol has no JS SDK so always use ethers package for JS code suggestions.

# CONTEXT
{augment_context}
{context}

# QUERY
```
{question}
```
"""
ANSWER_PROMPT = PromptTemplate(
    template=answer_prompt_template, input_variables=["context", "augment_context", "question"]
)


In [ ]:
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from typing import Optional, Type

complex_template = """
You are a web3 expert using Yield protocol and its services. {role_description}
Write me engaging and profound case study questions rooted in real life web3 user experience scenario using first-person pronoun (I) in the text, based on the provided documents.

# INSTRUCTIONS
- Compose {k} such long, qualitative and engaging real-life case study question that is arisen from the following document.
- Always ensure the questions are related to the document text and do not make up any information.
- Before you ask a question, make sure that you can also find evidences in the document in the form of sentences, phrases, or function descriptions that makes it answerable.
- Organize the results in lines, each question or each evidence takes a line. Do not enumerate the questions and evidences, just write them to lines.

# DOCUMENT
{document}

# EXAMPLE RESULT
evidence: <evidence 1>
evidence: <evidence 2>
evidence: <evidence 3>
...
question: <question 1>
question: <question 2>
question: <question 3>
...

# YOUR RESULT
"""

COMPLEX_QUESTION_PROMPT = PromptTemplate(
    template=complex_template, input_variables=["document", "role_description", "k"]
)

role_dict = {
    "doc" : "Here is a piece of document located at Yield Protocol's website and you have questions on it.",
    "addendum" : "Here is a piece of techinical document related to protocol operations. You are curious on how these code snippets work.",
    "governance" : "Here is a governance proposal proposing changes in the protocol. Remember that proposal numbers comes in chronologicial order, with larger number being recent proposals, and smaller number are old proposals.",
    "whitepaper" : "You are looking at its whitepaper and you want to talk to the authors of this paper to discuss about the details in this paper.",
    "yieldspace" : "You are looking at its newly published paper called 'yield space' and you want to talk to the authors of this paper to discuss about the details in this paper. Especially you noticed that this paper is heavily focused on mathematical formulas related to the concept of 'invariants' in decentralized finance. You are very interested in studying the maths behind this paper, pay special attention to the numeric relationships between objects in this passage."
}

# def ask_complex_question(document, role, k="3"):
#   complex_question_llm_chain = LLMChain(
#       llm=llm,
#       prompt=COMPLEX_QUESTION_PROMPT
#   )
#   print(f"Document:{document}")
#   print(f"Role: {role_dict[role]}")
#   print(f"k: {k}")
#   result = complex_question_llm_chain.run(document, role_description=role_dict[role], k=k)
#   return result

# question_writer = StructuredTool.from_function(
#         name="Question Writer",
#         func=ask_complex_question,
#         description="useful for when you need to ask engaging and high-quality questions on the given document in Yield Protocol related stuff.",
#     )

complex_answer_prompt_template = """
You are Yield Protocol's Web3 expert who is able to answer any user query on Yield protocol's documentation, code, whitepapers and many other such topics.

# INSTRUCTIONS
- The user's query will be wrapped in triple back ticks.
- Only answer the query using the provided context below which may include general information and code, do not make up any information.
- Aside from the documented context, there's also a list of evidences straightforwardly related to the question asked, if you found the fetched documents are contradicting or confusing, you can use the concise evidences to aid your navigation inside the context.
- If you want to support your answer from the context, quote the part in the document in your answer.
- If the query is related to code or integration, provide a step by step explanation on the process, use code suggestions whereever relevant and always use markdown format annotated with the language to show the code.
- If the query is related to mathematic formulas or mathematic proofs, explain in natural languguage in an easy-to-understand manner and use markdown/latex to compose math formulas if applicable.
- For code suggestions, use comments to explain each important concept, class, variable, function and parameter.
- However, if you could not find documented code reference in the context, do not attempt to deliniate your answer in code, instead, use natural language supported from the context retrieved.
- Yield Protocol has no JS SDK so always use ethers package for JS code suggestions.

# EVIDENCE
{evidence}

# CONTEXT
{augment_context}
{context}

# QUERY
```
{question}
```
"""
COMPLEX_ANSWER_PROMPT = PromptTemplate(
    template=complex_answer_prompt_template, input_variables=["context", "augment_context", "evidence", "question"]
)


In [ ]:
from google.colab import drive
import json


def answer(query, category, injected_doc, top_k=5, show_sources=False):
    display(Markdown(f"# Answering query:\n{query}"))

    llm_chain = LLMChain(
      llm=ChatOpenAI(temperature=0.1, model="gpt-4"),
      prompt=ANSWER_PROMPT
    )

    retriever=vector_db.as_retriever(search_type="mmr", search_kwargs = {
      'k': top_k,
      'filter': {'category': category}
    })

    context = retriever.get_relevant_documents(query)
    injected_context, injected_meta = injected_doc

    # context is multiple, we store them in the dataset for augmenting knowledge purpose (to inject to llama prompts)
    ret = {'question': query, 'answer': "", 'category': injected_meta['category'], 'contexts': [injected_context] + [con.page_content for con in context]}

    with get_openai_callback() as cb:
      ans = llm_chain.run(question=query, context=context, augment_context=injected_context)
      ret['answer'] = ans
      display(Markdown("# Final Answer"))
      display(Markdown(ans))
      print(cb)

      if show_sources:
        display(Markdown("### Sources"))
        display(Markdown(f"**[Source ORIGINAL]**"))
        display(Markdown(injected_context))
        if injected_meta['category'] != 'paper':
          display(Markdown(f"*File path: {injected_meta['file_path']}*"))

        for i, d in enumerate(context):
          display(Markdown(f"**[Source {i+1}]**"))
          display(Markdown(d.page_content))
          if d.metadata['category'] != 'paper':
            display(Markdown(f"*File path: {d.metadata['file_path']}*"))

      return ret


# Gets saved in the "tests/question_answering" directory
def save_qa_pair(file_name, qa_entities):
  with open(f"/content/drive/MyDrive/{file_name}", 'a+') as file:
    for qa in qa_entities:
      json.dump(qa, file)
      file.write("\n")


# integrated QA pipeline and save on disk on promise, one pipeline is a full chunk document db section (general, technical, whitepaper, etc.)
def gen_QA_pairs(chain, texts, metadatas, file_name):
  print(f"total texts: {len(texts)}")

  for i, t in enumerate(texts):
    with get_openai_callback() as cb:
      print(f"Working on text #{i+1}")
      qresult = chain.run(document=t)
      print(qresult)
      q_array = qresult.split('\n')
      print(f"Asked questions: {q_array}")
      print(cb)

      # attempt answer the question in the run freshly with context
      qa_array = []
      for q in q_array:
        qa_object = answer(q, category=metadatas[i]['category'], injected_doc=(t, metadatas[i]), show_sources=False)
        qa_array.append(qa_object)

      print(f"\nSaving QA pairs for text #{i+1}\n")
      save_qa_pair(file_name, qa_array)

In [ ]:
import json
import re

def complex_answer(query, category, injected_doc, evidences, top_k=5, show_sources=False):
    display(Markdown(f"# Answering query:\n{query}"))

    llm_chain = LLMChain(
      llm=ChatOpenAI(temperature=0, model="gpt-4"),
      prompt=COMPLEX_ANSWER_PROMPT
    )

    retriever=vector_db.as_retriever(search_type="mmr", search_kwargs = {
      'k': top_k,
    })

    context = retriever.get_relevant_documents(query)
    injected_context, injected_meta = injected_doc

    # context is multiple, we store them in the dataset for augmenting knowledge purpose (to inject to llama prompts)
    ret = {'question': query, 'answer': "", 'category': injected_meta['category'], 'contexts': [injected_context] + [con.page_content for con in context]}

    with get_openai_callback() as cb:
      ans = llm_chain.run(question=query, context=context, augment_context=injected_context, evidence=evidences)
      ret['answer'] = ans
      display(Markdown("# Final Answer"))
      display(Markdown(ans))
      print(cb)

      if show_sources:
        display(Markdown("### Sources"))
        display(Markdown(f"**[Source ORIGINAL]**"))
        display(Markdown(injected_context))
        if injected_meta['category'] != 'paper':
          display(Markdown(f"*File path: {injected_meta['file_path']}*"))

        for i, d in enumerate(context):
          display(Markdown(f"**[Source {i+1}]**"))
          display(Markdown(d.page_content))
          if d.metadata['category'] != 'paper':
            display(Markdown(f"*File path: {d.metadata['file_path']}*"))

      return ret

def gen_complex_QA_pairs(texts, chain, role, questions_per_text, metadatas, file_name):
  print(f"total texts: {len(texts)}")

  for i, t in enumerate(texts):
    with get_openai_callback() as cb:
      print(f"Working on text #{i+1}")
      display(Markdown(t))
      qresult = chain.run(document=t, role_description=role_dict[role], k=questions_per_text)
      print(qresult)
      lines = qresult.split('\n')

      q_array = []
      evids = []
      for line in lines:
        if line.split(':')[0].lower() == 'question':
          q_array.append("".join([segment for segment in line.split(':')[1:]]))
        elif line.split(':')[0].lower() == 'evidence':
          evids.append("".join([segment for segment in line.split(':')[1:]]))

      # q_array = json.loads(qresult)['questions']
      # evids = json.loads(qresult)['evidences']

      for q in q_array:
        print(f"Question asked: {q}")
      for e in evids:
        print(f"Evidences: {e}")
      print(cb)

      evid_str = "\n".join([e for e in evids])
      qa_array = []
      for q in q_array:
        qa_object = complex_answer(q, category=metadatas[i]['category'], injected_doc=(t, metadatas[i]), evidences=evid_str, show_sources=False)
        qa_array.append(qa_object)

      print(f"\nSaving QA pairs for text #{i+1}\n")
      save_qa_pair(file_name, qa_array)

def gen_governance_QA_pairs(texts, chain, metadatas, file_name, role='governance', questions_per_text="3"):
  with get_openai_callback() as cb:
      for i, t in enumerate(texts):
        pattern = r"^# YPP-\d{4}$" # YPP-0004, and so on.
        header = t.split('\n')[0]

        print(f"Working on text #{i+1}:{header}...")
        display(Markdown(t))
        qresult = chain.run(document=t, role_description=role_dict[role], k=questions_per_text)

        # parse the data right here programmatically and not lending this to LLM.
        print(qresult)
        lines = qresult.split('\n')

        q_array = []
        evids = []
        for line in lines:
          if line.split(':')[0].lower() == 'question':
            q_array.append("".join([segment for segment in line.split(':')[1:]]))
          elif line.split(':')[0].lower() == 'evidence':
            evids.append("".join([segment for segment in line.split(':')[1:]]))

        for j in range(len(q_array)):
          match_object = re.fullmatch(pattern, header)
          if match_object:
            q_array[j] = f"[{header[2:]}] {q_array[j]}" # make it like [YPP-0004] <question>
            print(f"Question asked: {q_array[j]}")
        for e in evids:
          print(f"Evidences: {e}")
        evid_str = "\n".join([e for e in evids])

        qa_array = []
        for q in q_array:
          qa_object = complex_answer(q, category=metadatas[i]['category'], injected_doc=(t, metadatas[i]), evidences=evid_str, show_sources=False)
          qa_array.append(qa_object)

        print(f"\nSaving QA pairs for text #{i+1}\n")
        save_qa_pair(file_name, qa_array)

In [ ]:
# drive.mount('/content/drive/')
# llm=ChatOpenAI(temperature=0.01, model="gpt-4")
# complex_question_llm_chain = LLMChain(llm=llm, prompt=COMPLEX_QUESTION_PROMPT)
# gen_governance_QA_pairs(governance_splits['all_splits'][501:], complex_question_llm_chain, governance_splits['all_metadatas'][501:], "governance_long_qa_3.jsonl")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Working on text #1:# YPP-0107...


# YPP-0107

Proposed 0x13f45f903c435e20c2f16b36b8da79062efcb20b5e34e33c1e396a16a6c5f9b2
+ npx hardhat run --network tenderly ./scripts/governance/add/addCollateral/addSensePTCollateral/../../../../../shared/approve.ts
Impersonated 0xd659565b84bcfcb23b02ee13e46cb51429f4558a
Proposal: 0x13f45f903c435e20c2f16b36b8da79062efcb20b5e34e33c1e396a16a6c5f9b2
Approving
Approved: 0x13f45f903c435e20c2f16b36b8da79062efcb20b5e34e33c1e396a16a6c5f9b2
advancing time by 172800 seconds (2 days) to 1677217301
+ npx hardhat run --network tenderly ./scripts/governance/add/addCollateral/addSensePTCollateral/../../../../../shared/execute.ts
Impersonated 0xC7aE076086623ecEA2450e364C838916a043F9a8
Proposal: 0x13f45f903c435e20c2f16b36b8da79062efcb20b5e34e33c1e396a16a6c5f9b2
Executing
Estimated gas: 919904 - ETH Balance: 1.0
Executed 0x13f45f903c435e20c2f16b36b8da79062efcb20b5e34e33c1e396a16a6c5f9b2
TxHash: 0x64174b9ffd0a8291c4db09a66d95ac9c420edf21de667488233e05b2ff4ef453

evidence: Proposed 0x13f45f903c435e20c2f16b36b8da79062efcb20b5e34e33c1e396a16a6c5f9b2
evidence: Impersonated 0xd659565b84bcfcb23b02ee13e46cb51429f4558a
evidence: Approving
evidence: Approved: 0x13f45f903c435e20c2f16b36b8da79062efcb20b5e34e33c1e396a16a6c5f9b2
evidence: advancing time by 172800 seconds (2 days) to 1677217301
evidence: Impersonated 0xC7aE076086623ecEA2450e364C838916a043F9a8
evidence: Executing
evidence: Estimated gas: 919904 - ETH Balance: 1.0
evidence: Executed 0x13f45f903c435e20c2f16b36b8da79062efcb20b5e34e33c1e396a16a6c5f9b2
evidence: TxHash: 0x64174b9ffd0a8291c4db09a66d95ac9c420edf21de667488233e05b2ff4ef453

question: As a user of the Yield protocol, I noticed that the proposal 0x13f45f903c435e20c2f16b36b8da79062efcb20b5e34e33c1e396a16a6c5f9b2 was approved. What changes does this proposal bring to the protocol and how will it affect my interaction with the protocol?
question: I saw that the execution of the proposal required an estimated gas of 919904 and the ETH bala

# Answering query:
[YPP-0107]  As a user of the Yield protocol, I noticed that the proposal 0x13f45f903c435e20c2f16b36b8da79062efcb20b5e34e33c1e396a16a6c5f9b2 was approved. What changes does this proposal bring to the protocol and how will it affect my interaction with the protocol?

# Final Answer

...

Tokens Used: 2575
	Prompt Tokens: 2306
	Completion Tokens: 269
Successful Requests: 1
Total Cost (USD): $0.08532000000000001


# Answering query:
[YPP-0113]  As a debtor in the June series, how will the automatic migration to the new June series with the same debt affect my repayment plan and what changes should I anticipate with the duplication of the June series on new contracts?

# Final Answer

The automatic migration of debtors in the June series to the new June series will not affect your repayment plan. The new June series is being duplicated on new contracts with a new series id, but otherwise identical to the old series. This means that the terms of your debt, including the amount and the repayment schedule, will remain the same as they were in the old June series.

The main change you should anticipate is that your debt will now be associated with the new June series contract, rather than the old one. This is a technical change that should not affect your experience as a debtor, as the functionality of the new contract is identical to the old one.

In addition, all series are being updated to use the Accumulator oracle at chi 0% and rate 5%, instead of relying on Compound for these values. This change is part of an effort to improve the accuracy and reliability of the protocol's interest rate calculations.

In summary, while there are some technical changes happening behind the scenes, your experience as a debtor should remain largely the same. You will still owe the same amount of debt, and it will still be due according to the same schedule.

Tokens Used: 2124
	Prompt Tokens: 1891
	Completion Tokens: 233
Successful Requests: 1
Total Cost (USD): $0.07071

Saving QA pairs for text #33



In [ ]:
# drive.mount('/content/drive/')
# llm=ChatOpenAI(temperature=0.01, model="gpt-4")
# complex_question_llm_chain = LLMChain(llm=llm, prompt=COMPLEX_QUESTION_PROMPT)
# gen_complex_QA_pairs(yieldspace_splits['all_splits'][24:27], chain=complex_question_llm_chain, role="yieldspace", questions_per_text="3", metadatas=yieldspace_splits['all_metadatas'][24:27], file_name="yieldspace_long_qa.jsonl")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
total texts: 3
Working on text #1


$$
\begin{gathered}
\frac{\partial}{\partial a} \ln \frac{a+b}{2}-\ln a \\
-\frac{b}{a b+a^{2}}
\end{gathered}
$$

This proves that the derivative of the invariant with respect to $t$ is less than or equal to 0 at all points, meaning that as time to maturity decreases, the invariant will either increase or stay the same.

evidence: The mathematical formula provided in the document.
evidence: The explanation that the derivative of the invariant with respect to $t$ is less than or equal to 0 at all points.
evidence: The statement that as time to maturity decreases, the invariant will either increase or stay the same.

question: How can I apply the mathematical formula provided in the Yield Space paper to optimize my investment strategy in the Yield protocol, considering the fact that as time to maturity decreases, the invariant will either increase or stay the same?
question: As a web3 user, how can I leverage the concept of 'invariants' in decentralized finance to manage my assets in the Yield protocol, especially in light of the mathematical relationships outlined in the Yield Space paper?
question: Given the mathematical formula and its implications in the Yield Space paper, how can I predict the behavior of my investments in the Yield protocol as time to maturity decreases?
Question asked:  How can I 

# Answering query:
 How can I apply the mathematical formula provided in the Yield Space paper to optimize my investment strategy in the Yield protocol, considering the fact that as time to maturity decreases, the invariant will either increase or stay the same?

# Final Answer

...

Tokens Used: 3661
	Prompt Tokens: 3130
	Completion Tokens: 531
Successful Requests: 1
Total Cost (USD): $0.12575999999999998


# Answering query:
 As a user of the Yield protocol, how would the mathematical relationships between $x$ and $y$ as discussed in the Yield Space paper affect my ability to predict and manage risk in my investments?

# Final Answer

The mathematical relationships between $x$ and $y$ as discussed in the Yield Space paper are used to model the behavior of the Yield Protocol's automated market maker. This model is designed to reduce the market impact suffered by traders of fyToken, especially for fyToken that is closer to maturity.

The derivative of the function that models the market maker's behavior is only equal to 0 when $x=y$ (given that $g t-\frac{t}{g} \neq 0)$. This means that the market impact is minimized when the amount of fyDai sold is equal to the amount of Dai received. This is a key point for managing risk, as it allows you to predict the market impact of your trades and adjust your strategy accordingly.

The second derivative of the function is positive when $x=y, x>0$, and $0<t<g<1$. This indicates that the point where $x=y$ is a minimum point, meaning that the market impact is always greater than or equal to 0. This further reinforces the idea that the market impact is minimized when the amount of fyDai sold is equal to the amount of Dai received.

In conclusion, understanding these mathematical relationships can help you predict the market impact of your trades and manage your risk accordingly. By trading in a way that minimizes market impact, you can potentially achieve better fyDai/Dai quotes and implied interest rates, thus maximizing the return on your investments.

Tokens Used: 2708
	Prompt Tokens: 2413
	Completion Tokens: 295
Successful Requests: 1
Total Cost (USD): $0.09008999999999999

Saving QA pairs for text #3



# Deduplicating known questions in the bank to leave only one in the dataset

In [ ]:
import openai
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Set up OpenAI API
openai.api_key = os.environ['OPENAI_API_KEY']

def get_embeddings(texts, model="text-embedding-ada-002"):
    emb_objs = openai.Embedding.create(input=texts, model=model)["data"]
    return [emb_objs[i]['embedding'] for i in range(len(emb_objs))]

# Function to calculate cosine similarity between two strings
def calculate_cosine_similarity(embedding1, embedding2):
    similarity = cosine_similarity(np.array(embedding1).reshape(1, -1),
                                   np.array(embedding2).reshape(1, -1))
    return similarity[0][0]

# Function to deduplicate similar questions
def deduplicate_questions(questions, threshold):
    duplicate_idxs = []
    embeddings = get_embeddings(questions)

    for i, emb1 in enumerate(embeddings): # if i is in duplicate_idxs, skip scanning i's duplicated mate.
      if i not in duplicate_idxs:
        is_duplicate = False
        duplicate_idx = -1
        for j, emb2 in enumerate(embeddings): # if i not marked duplicated, scan its duplicated mates, also skipping already marked duplicated ones
          if j > i and j not in duplicate_idxs:
            similarity = calculate_cosine_similarity(emb1, emb2)
            if similarity >= threshold: # j is a duplicate of i, break
                is_duplicate = True
                break

        if is_duplicate:
          duplicate_idxs.append(j)

    deduplicated_questions = [questions[i] for i in range(len(questions)) if i not in duplicate_idxs]
    return deduplicated_questions



#question_bank = documentation_questions + addendum_questions + whitepaper_questions + yieldspace_questions
question_bank = [
    "Hello, how are you?",
    "Hello how are yuou?",
    "Hi there!",
    "Hey, how's it going?",
    "Hello, how are you doing?",
    "Good day!",
]
threshold = 0.90  # Adjust this threshold based on your needs

deduplicated = deduplicate_questions(question_bank, threshold)

print("Deduplicated strings:")
for string in deduplicated:
    print(string)

In [ ]:
# # Discarded due to redundancy
# governance_proposals_question_prompt_template = """
# You are a web3 expert using Yield protocol and its services. Here is a governance proposal proposing changes in the protocol.
# Write me engaging and profound case study questions rooted in real life web3 user experience scenario using first-person pronoun (I) in the text, based on the provided documents.

# # INSTRUCTIONS
# - Compose 3 such long, qualitative and engaging real-life case study question that is arisen from the following proposal document
# - Always ensure the questions are related to the document text and do not make up any information.
# - When you ask a question, make sure that you can also find evidences in the document that makes it answerable.
# - The result should be wrapped in JSON format like {{"questions": []}}.

# # DOCUMENT
# {document}

# # RESULT"""

# GOV_QUESTION_BANK_PROMPT = PromptTemplate(
#     template=governance_proposals_question_prompt_template, input_variables=["document"]
# )

# governance_question_bank_llm_chain = LLMChain(
#     llm=ChatOpenAI(temperature=0.01, model="gpt-4"),
#     prompt=GOV_QUESTION_BANK_PROMPT
# )
# # prompt script for generating long whitepaper questions
# # how could it ask questions that can be answered in the
# whitepaper_question_prompt_template = """
# You are a web3 expert using Yield protocol and its services. You are looking at its whitepaper and you want to talk to the authors of this paper to discuss about the details in this paper.
# Write me engaging and profound case study questions rooted in real life web3 user experience scenario using first-person pronoun (I) in the text, based on the provided whitepaper pages.

# # INSTRUCTIONS
# - Compose 4 such long, qualitative and engaging real-life case study question that is arisen from the following whitepaper document.
# - Always ensure the questions are related to the document text and do not make up any information.
# - When you ask a question, make sure that you can also find evidences in the document that makes it answerable.
# - The result should be wrapped in JSON format like {{"questions": []}}.

# # DOCUMENT
# {document}

# # RESULT"""

# WP_QUESTION_BANK_PROMPT = PromptTemplate(
#     template = whitepaper_question_prompt_template, input_variables=["document"]
# )

# whitepaper_question_bank_llm_chain = LLMChain(
#     llm=ChatOpenAI(temperature=0.01, model="gpt-4"),
#     prompt=WP_QUESTION_BANK_PROMPT
# )
# yieldspace_question_prompt_template = """
# You are a web3 expert using Yield protocol and its services. You are looking at its newly published paper called 'yield space' and you want to talk to the authors of this paper to discuss about the details in this paper.
# Especially you noticed that this paper is heavily focused on mathematical formulas related to the concept of 'invariants' in decentralized finance.
# You are very interested in studying the maths behind this paper, pay special attention to the numeric relationships between objects in this passage.
# Write me engaging and profound case study questions rooted in web3 finance researcher experience using first-person pronoun (I) in the text, based on the provided paper pages.

# # INSTRUCTIONS
# - Compose 4 such long, qualitative and engaging real-life case study question that is arisen from the following whitepaper document.
# - Always ensure the questions are related to the document text and do not make up any information.
# - When you ask a question, make sure that you can also find evidences in the document that makes it answerable.
# - The result should be wrapped in JSON format like {{"questions": []}}.

# # DOCUMENT
# {document}

# # RESULT"""

# YS_QUESTION_BANK_PROMPT = PromptTemplate(
#     template = yieldspace_question_prompt_template, input_variables=["document"]
# )

# yieldspace_question_bank_llm_chain = LLMChain(
#     llm=ChatOpenAI(temperature=0.01, model="gpt-4"),
#     prompt=YS_QUESTION_BANK_PROMPT
# )